In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import hipp
from getpass import getpass
import glob
import cv2
# import os
# import sys
# import pandas as pd
# import json
# import requests
# import matplotlib.pyplot as plt


## Login

In [10]:
apiKey = hipp.dataquery.EE_login(input(), getpass())

## Inputs

In [11]:
bounds = (-122, 49, -121.5, 48.5)
ee_project_name = 'LK000'
year = 1950
month = 9
day = 2
output_directory    = './'
ee_query_max_results   = 10 #do more than 1 because you may get a calibration file
ee_query_label = 'test_download' #try to use the same one for consistent behavior from the EE API

## Query EE

and filter results by the project name you are interested in (or look at the entire ee_results_df first)

In [17]:
ULLON, ULLAT, LRLON, LRLAT = bounds

ee_results_df = hipp.dataquery.EE_pre_select_images(
    apiKey,
    xmin = LRLON,
    ymin = LRLAT,
    xmax = ULLON,
    ymax = ULLAT,
    startDate = f"{year}-{month}-{day}",
    endDate = f"{year}-{month}-{day}",
    maxResults   = ee_query_max_results
)

ee_results_df = ee_results_df[ee_results_df['project'] == ee_project_name]

Max records requested: 10

Bounds:
xmin -121.5 
ymin 48.5 
xmax -122 
ymax 49

Time range: 1950-9-2 to 1950-9-2

Records returned: 10

maxResults set to: 10 
Increase this parameter to obtain additional records. API max 50,000.


In [18]:
ee_results_df

,entityId,agency,recordingTechnique,project,roll,frame,acquisitionDate,hi_res_available,imageType,quality,...,centerLat,centerLon,NWlat,NWlon,NElat,NElon,SElat,SElon,SWlat,SWlon
0,AR1LK0000010027,1,1,LK000,000001,27,1950/09/02,Y,24,8,...,48.573584,-121.437615,48.623248,-121.504815,48.618177,-121.362778,48.524012,-121.370541,48.529071,-121.512317
1,AR1LK0000010028,1,1,LK000,000001,28,1950/09/02,Y,24,8,...,48.575187,-121.501556,48.623945,-121.570252,48.620772,-121.428082,48.526521,-121.432989,48.529686,-121.574895
2,AR1LK0000010029,1,1,LK000,000001,29,1950/09/02,Y,24,8,...,48.575028,-121.565574,48.623344,-121.634977,48.621081,-121.492767,48.526803,-121.496303,48.529058,-121.638249
3,AR1LK0000010030,1,1,LK000,000001,30,1950/09/02,Y,24,8,...,48.575772,-121.630216,48.624274,-121.699326,48.621630,-121.557126,48.527361,-121.561236,48.529997,-121.703172
4,AR1LK0000010031,1,1,LK000,000001,31,1950/09/02,Y,24,8,...,48.578620,-121.688865,48.626502,-121.758954,48.625126,-121.616705,48.530828,-121.618907,48.532199,-121.760892
5,AR1LK0000010032,1,1,LK000,000001,32,1950/09/02,Y,24,8,...,48.576990,-121.759778,48.624930,-121.829778,48.623439,-121.687534,48.529141,-121.689910,48.530627,-121.831890
6,AR1LK0000010033,1,1,LK000,000001,33,1950/09/02,Y,24,8,...,48.575692,-121.830591,48.623372,-121.900991,48.622407,-121.758739,48.528101,-121.760323,48.529062,-121.902311
7,AR1LK0000010034,1,1,LK000,000001,34,1950/09/02,Y,24,8,...,48.576073,-121.893026,48.623080,-121.964450,48.623468,-121.822189,48.529155,-121.821736,48.528766,-121.963732
8,AR1LK0000010035,1,1,LK000,000001,35,1950/09/02,Y,24,8,...,48.573384,-121.955782,48.620869,-122.026483,48.620301,-121.884229,48.525989,-121.885215,48.526554,-122.027204
9,AR1LK0000010036,1,1,LK000,000001,36,1950/09/02,Y,24,8,...,48.573993,-122.020103,48.621495,-122.090778,48.620892,-121.948522,48.526580,-121.949561,48.527180,-122.091553


## Select the first image for download and download it

In [19]:
ee_results_df = ee_results_df.head(1)

In [23]:
images_directory, calibration_reports_directory = hipp.dataquery.EE_download_images_to_disk(
    apiKey,
    ee_results_df['entityId'].tolist(),
    ee_query_label,
    output_directory
)
single_image_file = glob.glob(os.path.join(images_directory,'*.tif'))[0]

## Read and write with opencv (cv2) to correct the pixel order 

This appears to be necessary for the HIPP preprocessing tools to work. All EE images I have encountered have had this issue, but this step might be optional for yet-to-be-examined datasets.

In [26]:
image = cv2.imread(single_image_file)
cv2.imwrite(single_image_file, image)

True

## Create fiducial templates from images

Note that the optional parameter buffer_distance should be tuned to the dataset you are using.

In [35]:
fiducial_template_directory = os.path.join(output_directory, 'fiducials')

hipp.core.create_midside_fiducial_proxies_template(
    single_image_file,
    fiducial_template_directory,
    buffer_distance = 250 #default value

)

Select inner most point to crop from for midside fiducial marker proxies,
in order from Left - Top - Right - Bottom.
Launching server at http://localhost:60718


OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
